In [8]:
import sympy as sp
from IPython.display import Markdown, Latex, display
x, y, z = sp.symbols('x y z')
X = sp.Matrix([x, y, z])

In [16]:
def LatexEnv(name, core, c=None):
    if c==None: c=''
    else: c=f'{{{c}}}'
    return f'\\begin{{{name}}}{c} {core} \\end{{{name}}}'
LatexEnv('cases', 'abc')

'\\begin{cases} abc \\end{cases}'

In [14]:
def LatexArgs(name, *args):
    return f'\\{name}'+''.join(f'{{{arg}}}' for arg in args)
LatexArgs('frac','a','n')

'\\frac{a}{n}'

In [142]:
def LatexEq(steps):
    def _tex(n):
        if type(n) is str: return n
        else: return sp.latex(n)
        
    return ' = '.join([_tex(s) for s in steps])

LatexEq(DetCalc_3x3_process(A, name='A'))

'\\text{det(A)} = - (1 \\cdot 3 \\cdot 5 + 3 \\cdot 5 \\cdot 4 + 3 \\cdot 5 \\cdot 5) + \\left(1 \\cdot 5 \\cdot 5 + 4 \\cdot 3 \\cdot 3 + 5 \\cdot 5 \\cdot 3\\right) = - (15 + 60 + 75) + \\left(25 + 36 + 75\\right) = -150 + 136 = -14'

In [153]:
def latexy(f):
    return f'${f}$'

In [118]:
def AB_to_system(M1, M2):
    #given AX, return equations needed
    return (sp.Eq(a,b) for a,b in zip(M1, M2))
                                        
equations = list(AB_to_system(A*X, B))
equations

[Eq(x + 3*y + 3*z, 4), Eq(5*x + 5*y + 3*z, 4), Eq(4*x + 5*y + 5*z, 9)]

In [10]:
def system_to_latex(equations):
    latexify = [sp.latex(n) for n in equations]
    core = ' \\\\ '.join(latexify)
    return LatexEnv('cases', core)

result = system_to_latex(equations)
result

'\\begin{cases} x + 3 y + 3 z = 4 \\\\ 5 x + 5 y + 3 z = 4 \\\\ 4 x + 5 y + 5 z = 9 \\end{cases}'

In [11]:
def AB_to_latex(A, B):
    return system_to_latex(AB_to_system(A, B))
AB_to_latex(A*X, B)

'\\begin{cases} x + 3 y + 3 z = 4 \\\\ 5 x + 5 y + 3 z = 4 \\\\ 4 x + 5 y + 5 z = 9 \\end{cases}'

In [72]:
display(Markdown(system_to_latex(AX_to_system(A*X, B))))

\begin{cases} x + 3 y + 3 z = 4 \\ 5 x + 5 y + 3 z = 4 \\ 4 x + 5 y + 5 z = 9 \end{cases}

In [203]:
def DetCalc_3x3_process(M, name='A'):
    step0 = LatexArgs('text', f'det({name})')
    detcore = "\\\\".join([f"{M[0]}&{M[1]}&{M[2]}", f"{M[3]}&{M[4]}&{M[5]}", f"{M[6]}&{M[7]}&{M[8]}"])
    step1 = f'\\left|{LatexEnv("array", detcore, "ccc")}\\right|'
    step2 = sp.Add(sp.Add(sp.Mul(M[0],M[4],M[8], evaluate=False),
                                   sp.Mul(M[3],M[7],M[2], evaluate=False),
                                   sp.Mul(M[6],M[1],M[5], evaluate=False), evaluate=False),
                  sp.Mul(-1, sp.Add(sp.Mul(M[2],M[4],M[6], evaluate=False),
                                    sp.Mul(M[1],M[3],M[8], evaluate=False),
                                    sp.Mul(M[0],M[5],M[7], evaluate=False), 
                                    evaluate=False), evaluate=False), evaluate=False)
    step3 = sp.Add(sp.Add(M[0]*M[4]*M[8], M[3]*M[7]*M[2], M[6]*M[1]*M[5], evaluate=False),
                  sp.Mul(-1, sp.Add(M[2]*M[4]*M[6], M[1]*M[3]*M[8], M[0]*M[5]*M[7], evaluate=False), 
                         evaluate=False), evaluate=False)
    step4 = sp.Add(M[0]*M[4]*M[8]+M[3]*M[7]*M[2]+M[6]*M[1]*M[5],
                  -M[2]*M[4]*M[6]-M[1]*M[3]*M[8]-M[0]*M[5]*M[7], evaluate=False)
    step5 = M[0]*M[4]*M[8]+M[3]*M[7]*M[2]+M[6]*M[1]*M[5]-M[2]*M[4]*M[6]-M[1]*M[3]*M[8]-M[0]*M[5]*M[7]
    return step0, step1, step2, step3, step4, step5

DetCalc_3x3_process(A, name='A')

('\\text{det(A)}',
 '\\left|\\begin{array}{ccc} 1&3&3\\\\5&5&3\\\\4&5&5 \\end{array}\\right|',
 -(1*3*5 + 3*5*4 + 3*5*5) + 1*5*5 + 4*3*3 + 5*5*3,
 -(15 + 60 + 75) + 25 + 36 + 75,
 -150 + 136,
 -14)

## Let's run Cramer now!
Išspręsti Kramerio metodu lygčių sistemą, kurios koeficientų matrica yra A ir laisvųjų narių stulpelis yra B.

$A = \left(\begin{array}{ccc} 1&3&3 \\ 5&5&3 \\ 4&5&5 \end{array}\right)$; $B = \left(\begin{array}{c} 4\\ 4 \\ 9 \end{array}\right)$

$X = \left(\begin{array}{c} x \\ y \\ z \end{array}\right)$

Tada, jei moki daugybą, gali įsitikinti, kad:

$AX = \left(\begin{array}{c} x+3y+3z \\ 5x+5y+3z \\ 4x+5y+5z \end{array}\right)$

Prilyginam $AX=B$

$\left(\begin{array}{c} x+3y+3z \\ 5x+5y+3z \\ 4x+5y+5z \end{array}\right) = \left(\begin{array}{c} 4\\ 4 \\ 9 \end{array}\right)$

Tai buvo kitas būdas užsirašyti lygčių sistemai

$\begin{cases} x+3y+3z=4 \\ 5x+5y+3z=4 \\ 4x+5y+5z=9 \end{cases}$



In [299]:
#A = sp.Matrix([[5, 4, 3], [2, 3, 0], [0, 3, 4]])
#B = sp.Matrix([[30], [11], [11]])

In [303]:
A = sp.Matrix([[1, 3, 3], [5, 5, 3], [4, 5, 5]])
B = sp.Matrix([[4], [4], [9]])

In [313]:
solution = []
results = []
steps = DetCalc_3x3_process(A.copy(), name=latexy('D'))
solution.append(latexy(LatexEq(steps)))
results.append(steps[-1])
for i in range(3):
    D = A.copy()
    D = D.col_insert(i, B)
    D.col_del(i+1)
    steps = DetCalc_3x3_process(D, name=latexy(f'D_{i+1}'))
    solution.append(latexy(LatexEq(steps)))
    results.append(steps[-1])

D, D1, D2, D3 = results
data0 = 'A = ' + latexy(sp.latex(A)) +'; '
data1 = 'B = ' + latexy(sp.latex(B))
kramer = ' '.join(solution)

display(Markdown(data0))
display(Markdown(data1))
display(Markdown('Gaunasi sistema:'+ system_to_latex(AB_to_system(A*X, B))))
display(Markdown(kramer))
display(Markdown(f"$x=\\displaystyle{LatexArgs('frac', 'D_1', 'D')} = {LatexArgs('frac', D1, D)} = {D1/D}$"))
display(Markdown(f"$y=\\displaystyle{LatexArgs('frac', 'D_2', 'D')} = {LatexArgs('frac', D2, D)} = {D2/D}$"))
display(Markdown(f"$z=\\displaystyle{LatexArgs('frac', 'D_3', 'D')} = {LatexArgs('frac', D3, D)} = {D3/D}$"))

A = $\left[\begin{matrix}1 & 3 & 3\\5 & 5 & 3\\4 & 5 & 5\end{matrix}\right]$; 

B = $\left[\begin{matrix}4\\4\\9\end{matrix}\right]$

Gaunasi sistema:\begin{cases} x + 3 y + 3 z = 4 \\ 5 x + 5 y + 3 z = 4 \\ 4 x + 5 y + 5 z = 9 \end{cases}

$\text{det($D$)} = \left|\begin{array}{ccc} 1&3&3\\5&5&3\\4&5&5 \end{array}\right| = - (1 \cdot 3 \cdot 5 + 3 \cdot 5 \cdot 4 + 3 \cdot 5 \cdot 5) + \left(1 \cdot 5 \cdot 5 + 4 \cdot 3 \cdot 3 + 5 \cdot 5 \cdot 3\right) = - (15 + 60 + 75) + \left(25 + 36 + 75\right) = -150 + 136 = -14$ $\text{det($D_1$)} = \left|\begin{array}{ccc} 4&3&3\\4&5&3\\9&5&5 \end{array}\right| = - (3 \cdot 4 \cdot 5 + 4 \cdot 3 \cdot 5 + 3 \cdot 5 \cdot 9) + \left(4 \cdot 5 \cdot 3 + 9 \cdot 3 \cdot 3 + 4 \cdot 5 \cdot 5\right) = - (60 + 60 + 135) + \left(60 + 81 + 100\right) = -255 + 241 = -14$ $\text{det($D_2$)} = \left|\begin{array}{ccc} 1&4&3\\5&4&3\\4&9&5 \end{array}\right| = - (1 \cdot 3 \cdot 9 + 3 \cdot 4 \cdot 4 + 4 \cdot 5 \cdot 5) + \left(1 \cdot 4 \cdot 5 + 4 \cdot 4 \cdot 3 + 5 \cdot 9 \cdot 3\right) = - (27 + 48 + 100) + \left(20 + 48 + 135\right) = -175 + 203 = 28$ $\text{det($D_3$)} = \left|\begin{array}{ccc} 1&3&4\\5&5&4\\4&5&9 \end{array}\right| = - (1 \cdot 4 \cdot 5 + 4 \cdot 5 \cdot 4 + 3 \cdot 5 \cdot 9) + \left(1 \cdot 5 \cdot 9 + 4 \cdot 3 \cdot 4 + 5 \cdot 5 \cdot 4\right) = - (20 + 80 + 135) + \left(45 + 48 + 100\right) = -235 + 193 = -42$

$x=\displaystyle\frac{D_1}{D} = \frac{-14}{-14} = 1$

$y=\displaystyle\frac{D_2}{D} = \frac{28}{-14} = -2$

$z=\displaystyle\frac{D_3}{D} = \frac{-42}{-14} = 3$

### Bugs
Can't set order of summands while counting determinant of 3*3 matrix

In [4]:
import numpy as np
A = np.array([[5, 4, 3], [2, 3, 0], [0, 3, 4]], dtype=int)
B = np.array([[5, 4], [2, 3], [0, 3]], dtype=int)
print(A)
print(B)

[[5 4 3]
 [2 3 0]
 [0 3 4]]
[[5 4]
 [2 3]
 [0 3]]


In [9]:
[[[sp.Mul(x,y,evaluate=False) for x,y in zip(m,n)] for n in np.transpose(B)] for m in A]

[[[5*5, 4*2, 3*0], [5*4, 4*3, 3*3]],
 [[2*5, 3*2, 0*0], [2*4, 3*3, 0*3]],
 [[0*5, 3*2, 4*0], [0*4, 3*3, 4*3]]]